In [1]:
import numpy as np
import tensorflow as tf

In [2]:
print(tf.__version__)

2.0.0-beta0


## Gradient tapes

In [3]:
x = tf.ones(shape=(2, 2))

with tf.GradientTape() as t:
    t.watch(x)
    y = tf.reduce_sum(x)
    z = tf.multiply(y, y)
    
dz_dx = t.gradient(z, x)

print(dz_dx.numpy())

[[8. 8.]
 [8. 8.]]


In [4]:
# intermediate gradient
x = tf.ones(shape=(2, 2))

with tf.GradientTape() as t:
    t.watch(x)
    y = tf.reduce_sum(x)
    z = tf.multiply(y, y)
    
dz_dy = t.gradient(z, y)

print(dz_dy.numpy())

8.0


In [5]:
# これはエラーになる
# .gradientが一度呼ばれるとGradientTapeは解放される
x = tf.ones(shape=(2, 2))

with tf.GradientTape() as t:
    t.watch(x)
    y = tf.reduce_sum(x)
    z = tf.multiply(y, y)
    
dz_dx = t.gradient(z, x)    
dz_dy = t.gradient(z, y)

RuntimeError: GradientTape.gradient can only be called once on non-persistent tapes.

In [6]:
# 複数回.gradientを呼ぶ場合はpersistent=Trueとする
# リソースは手動で解放
x = tf.ones(shape=(2, 2))

with tf.GradientTape(persistent=True) as t:
    t.watch(x)
    y = tf.reduce_sum(x)
    z = tf.multiply(y, y)
    
dz_dx = t.gradient(z, x)    
dz_dy = t.gradient(z, y)

del t

## Higher-order gradients

In [7]:
x = tf.Variable(2.0)

with tf.GradientTape() as t:
    with tf.GradientTape() as t2:
        y = x * x
    
    dy_dx = t2.gradient(y, x)
d2y_d2x = t.gradient(dy_dx, x)

print(dy_dx)
print(d2y_d2x)

tf.Tensor(4.0, shape=(), dtype=float32)
tf.Tensor(2.0, shape=(), dtype=float32)


## .watchについて

cf. tf.GradientTapeのドキュメント（argsのwatch_accessed_variablesの部分など）

デフォルトではtrainableなVariableを自動でwatchしている。細かいコントロールをしたい時は`watch_accessed_variables=False`とする。